# XOR utilizando redes neuronales con Tensorflow y Keras.

## Cargamos las bibliotecas a utilizar

In [3]:
# TensorFlow y tf.keras
import tensorflow as tf
from tensorflow import keras

import numpy as np
from keras.models import Sequential
from keras.layers.core import Dense

ModuleNotFoundError: No module named 'tensorflow'

## Construimos el conjunto de datos

In [ ]:
# Los cuatro estados de la compuerta XOR
training_data = np.array([[0,0],[0,1],[1,0],[1,1]], "float32")

# Los cuatro resultados esperados en el mismo orden
target_data = np.array([[0],[1],[1],[0]], "float32")

## Construimos el modelo de redes neuronales

In [ ]:
model = Sequential()
model.add(Dense(16, input_dim=2, activation='relu'))
model.add(Dense(1, activation='sigmoid'))

model.compile(loss='mean_squared_error',
              optimizer='adam',
              metrics=['binary_accuracy'])

## Entrenamos el modelo

In [ ]:
model.fit(training_data, target_data, nb_epoch=500, verbose=2)

## Realizamos las predicciones

In [ ]:
print model.predict(training_data).round()